In [7]:
import sys
sys.path.append('/app/')  # noqa

from jina import Document, DocumentArray, Flow, Executor, requests
from pathlib import Path
import argparse
from loopa.executors.clip import CLIPTextEncoder
from loopa.executors.index import FaissIndexer, IndexMerger
from config import IMAGE_DIR, JINA_EMBS_COLLECTION_NAME, EMBEDDER_PARAMS
import torch

ModuleNotFoundError: No module named 'loopa.executors'

In [5]:
!ls /app/loopa

Dockerfile.cuda  docker-compose.jina.yml       notebooks
README.md	 docker-compose.jupyter.yml    requirements.txt
apps		 docker-compose.search_ui.yml
bo4ka		 loopa


In [ ]:


ROOT_DIR = Path('/data/cache/streamlit_search')

MODEL_OPTIONS = {
    # name : flow_branch_prefix
    "Clip": FLOW_BRANCH_CLIP_PREFIX,
}


def get_url(filename):
    return f'{CDN_SERVER_URL}/data/cache/streamlit_search/{filename}'


def get_image_url(image):
    filename = f'{uuid4()}.jpg'
    dst_file = ROOT_DIR/filename
    image.convert('RGB').save(dst_file)
    return get_url(filename)


def to_base64_str(image):
    buffered = BytesIO()
    image.save(buffered, format="JPEG")
    img_str = base64.b64encode(buffered.getvalue())
    return img_str


def search(text, models_to_use, top_k=10):

    parent_id = str(uuid4())

    print('text', text)

    models = []
    for _, value in models_to_use.items():
        models.append(MODEL_OPTIONS[value])

    models_str = "|".join(models)

    payload = {
        "data": [
            {"text": text}
        ],
        "exec_endpoint": "/search",
        "parameters": {
            "top_k": top_k,
        },
        "target_peapod": f"{models_str}|merger"
    }

    # print(JINA_SEARCH_ENDPOINT)
    # print("payload", payload)

    r = requests.post(JINA_SEARCH_ENDPOINT, json=payload)
    response = r.json()

    result = {}
    req_docs = response['data']['docs']
    for req_doc in req_docs:
        if req_doc.get("tags"):
            if req_doc["tags"].get("indexer_tag"):
                prefix = req_doc["tags"]["indexer_tag"].split("_")[0]
                if prefix in FLOW_BRANCH_PREFIXES:
                    result[prefix] = req_doc["matches"]

    return result
